In [1]:
import speech_recognition as sr
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from pywebio import start_server
from pywebio.input import input, textarea
from pywebio.output import put_text, put_html, put_buttons, put_image
from pywebio.session import defer_call
import matplotlib.pyplot as plt
import numpy as np
from pywebio.output import put_text, put_html, put_buttons, put_image, put_table
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle



2024-04-29 22:24:34.362824: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 22:24:35.008485: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 22:24:38.874680: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/pratyasa/NLP_EMOTION_DEPICTOR/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook imp

In [2]:
# Load tokenizer and model configuration
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")
config = AutoConfig.from_pretrained("fine_tuned_model")


In [3]:
# Load model
model = TFAutoModelForSequenceClassification.from_pretrained("fine_tuned_model", config=config)

2024-04-29 22:24:51.261842: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 22:24:51.263013: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on,

In [4]:

# Initialize recognizer
recognizer = sr.Recognizer()


In [5]:
# Variable to keep track of recording state
is_recording = False


In [6]:
# # Initialize lists to store conversation outputs and distraction levels
conversation_outputs = []
predicted_emotions = []
probability_distributions = []
distraction_levels = []


In [7]:
# Function to perform speech recognition with customizable parameters
def recognize_speech(sample_rate=44100, chunk_size=1024):
    global is_recording
    # Start recording if not already recording
    if not is_recording:
        is_recording = True
        put_text("Speak", "Speak something...")
        
        # Adjust microphone sample rate and chunk size
        with sr.Microphone(sample_rate=sample_rate, chunk_size=chunk_size) as source:
            # Adjust energy threshold dynamically
            recognizer.adjust_for_ambient_noise(source)
            # Listen for audio input
            audio = recognizer.listen(source)

        try:
            # Recognize speech using Google Speech Recognition
            user_input = recognizer.recognize_google(audio)
            conversation_outputs.append(user_input)  # Save conversation output
            handle_input(user_input)
            generate_probability_distribution_graph() 
        except sr.UnknownValueError:
            put_text("Sorry, I could not understand what you said.")
        except sr.RequestError as e:
            put_text("Sorry, I could not request results from Google Speech Recognition service; {0}".format(e))
        
        is_recording = False
        

In [8]:
#Function to handle input and predict emotion
def handle_input(user_input):
    global probability_distributions
    put_text("User:", user_input)
    # Tokenize the text
    input_encoded = tokenizer(user_input, return_tensors='tf')

    # Perform inference
    outputs = model(input_encoded)

    # Get predicted label and probabilities
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=1).numpy()[0]
    pred = tf.argmax(logits, axis=1).numpy()
    predicted_label = pred[0]
    predicted_emotions.append(predicted_label)
    probability_distributions.append(probabilities)
     # Display predicted emotion and probabilities
    emojis = {
        'sadness': '😔',
        'joy': '🤗',
        'love': '❤️',
        'anger': '😠',
        'fear': '😨😱',
        'surprise': '😮'
    }
    labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
    predicted_emoji = emojis[labels[predicted_label]]

    put_text("Chatbot:", f"The predicted emotion is: {labels[predicted_label]}{predicted_emoji}")


    # Calculate distraction level (for demonstration purposes, a random value is used)
    distraction_level = np.random.uniform(0, 1)
    distraction_levels.append(distraction_level)
    

In [9]:

# Function to generate and display the probability distribution bar graph

def generate_probability_distribution_graph():
    global probability_distributions
    # Plot probability distribution bar graph
    
    plt.rcParams['axes.facecolor'] = 'white'
    plt.figure(figsize=(10, 6))
    labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
    probs = probability_distributions[-1] 
    colors = ['#9FB9C7', '#3D657A', '#C8E8FA', '#7DCFFB', '#677881', '#5B96A9']
    bars = plt.bar(labels, probs, color=colors)
    emoji_dict = {
        'sadness': '😔',
        'joy': '🤗',
        'love': '❤️',
        'anger': '😠',
        'fear': '😨😱',
        'surprise': '😮'
    }

    x_positions = [bar.get_x() + bar.get_width() / 2 for bar in bars]  # Center of each bar
    y_positions = [bar.get_height() + 0.1 for bar in bars]  # Slightly above the bar
    for x, y, label in zip(x_positions, y_positions, labels):
        plt.text(x, y, emoji_dict[label], ha='center', va='bottom', fontsize=20)

    plt.xlabel('Emotion')
    plt.ylabel('Probability')
    plt.title('Probability Distribution of Emotions')
    plt.grid(False)
    plt.tight_layout()

    # Save plot as image
    plt.savefig('probability_distribution_graph.png')
    plt.close()
    

    
    # Display the generated image
    put_image(open('probability_distribution_graph.png', 'rb').read(), width='80%')
    


In [10]:
# Function to generate and display the distraction graph
#

def generate_distraction_graph():
    global distraction_levels
    # Plot distraction graph
    plt.figure(figsize=(10, 6))
    turns = range(1, len(distraction_levels) + 1)
    plt.plot(turns, distraction_levels, marker='o', linestyle='-',color='r')
    plt.xlabel('Turns')
    plt.ylabel('Distraction Level')
    plt.title('Distraction Throughout Conversation')
    plt.grid(False)
    plt.tight_layout()

    # Save plot as image
    plt.savefig('distraction_graph.png')
    plt.close()

    # Display the generated image
    put_image(open('distraction_graph.png', 'rb').read(), width='80%')


In [11]:
#PyWebIO chatbot interface
def chatbot_app():
    put_html("<div style='background-color: white; padding: 20px;'> <h1>Emotion Depicter</h1> </div>")
    
    def handle_text_input():
        user_input = input("User:", placeholder="Type here...")
        conversation_outputs.append(user_input)  # Save conversation output
        handle_input(user_input)
        generate_probability_distribution_graph()

    def handle_speech_input():
        recognize_speech()
        generate_probability_distribution_graph()

    def generate_graph():
        generate_distraction_graph()
       

    put_buttons(['Please Write', '🎤', 'Generate Distraction Graph'], onclick=[handle_text_input, handle_speech_input, generate_graph])

In [12]:
if __name__ == "__main__":
    start_server(chatbot_app, port=8086)

Running on all addresses.
Use http://172.16.144.103:8086/ to access the application


RuntimeError: This event loop is already running

/tmp/ipykernel_11261/360626166.py:31: UserWarning: Glyph 129303 (\N{HUGGING FACE}) missing from current font.
/tmp/ipykernel_11261/360626166.py:34: UserWarning: Glyph 129303 (\N{HUGGING FACE}) missing from current font.
